In [21]:
import pandas as pd # Import the pandas library for data manipulation

orders_df = pd.read_csv('orders.csv') # Load 'orders.csv' into a DataFrame
orders_df.head() # Display the first few rows of the DataFrame

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [22]:
users_df = pd.read_json('users.json') # Load 'users.json' into a DataFrame
users_df.head() # Display the first few rows of the DataFrame

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [23]:
import sqlite3 # Import sqlite3 for database operations
import io # Import io module (not strictly needed here for file open, but was in original)

conn = sqlite3.connect(':memory:') # Create an in-memory SQLite database connection

with open('restaurants.sql', 'r') as f: # Open the SQL file for reading
    sql_script = f.read() # Read the entire SQL script content

conn.executescript(sql_script) # Execute all SQL commands in the in-memory database

restaurants_df = pd.read_sql_query('SELECT * FROM restaurants', conn) # Load data from 'restaurants' table into a DataFrame

restaurants_df.head() # Display the first few rows of the DataFrame

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [24]:
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left') # Perform a left merge on 'orders_df' and 'users_df' using 'user_id'
merged_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left') # Perform another left merge with 'restaurants_df' using 'restaurant_id'
merged_df.head() # Display the first few rows of the final merged DataFrame

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [25]:
merged_df.to_csv('final_food_delivery_dataset.csv', index=False) # Save the merged DataFrame to a CSV file, excluding the index
print("Merged DataFrame saved to 'final_food_delivery_dataset.csv'") # Print a confirmation message

Merged DataFrame saved to 'final_food_delivery_dataset.csv'


In [26]:
gold_members_df = merged_df[merged_df['membership'] == 'Gold'] # Filter DataFrame for 'Gold' members
city_revenue = gold_members_df.groupby('city')['total_amount'].sum() # Group by city and sum 'total_amount'
highest_revenue_city = city_revenue.idxmax() # Find the city with the maximum revenue
max_revenue = city_revenue.max() # Get the maximum revenue value

print(f"City with the highest revenue from Gold members: {highest_revenue_city}") # Print the city name
print(f"Total revenue from Gold members in {highest_revenue_city}: {max_revenue:.2f}") # Print the total revenue for that city

City with the highest revenue from Gold members: Chennai
Total revenue from Gold members in Chennai: 1080909.79


In [27]:
cuisine_avg_order_value = merged_df.groupby('cuisine')['total_amount'].mean() # Group by cuisine and calculate the mean of 'total_amount'
highest_avg_cuisine = cuisine_avg_order_value.idxmax() # Find the cuisine with the highest average order value
max_avg_value = cuisine_avg_order_value.max() # Get the highest average value

print(f"Cuisine with the highest average order value: {highest_avg_cuisine}") # Print the cuisine name
print(f"Highest average order value: {max_avg_value:.2f}") # Print the highest average order value

Cuisine with the highest average order value: Mexican
Highest average order value: 808.02


In [28]:
user_total_orders = merged_df.groupby('user_id')['total_amount'].sum() # Group by user_id and sum their total order amounts
users_over_1000 = user_total_orders[user_total_orders > 1000] # Filter for users with total orders greater than 1000
count_users_over_1000 = users_over_1000.count() # Count the number of such distinct users

print(f"Number of distinct users with a total order value greater than \u20b91000: {count_users_over_1000}") # Print the final count

Number of distinct users with a total order value greater than ₹1000: 2544


In [29]:
bins = [3.0, 3.6, 4.1, 4.6, 5.1]
labels = ['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0']

merged_df['rating_range'] = pd.cut(merged_df['rating'], bins=bins, labels=labels, right=False)

revenue_by_rating_range = merged_df.groupby('rating_range')['total_amount'].sum()
highest_revenue_range = revenue_by_rating_range.idxmax()
max_revenue_rating_range = revenue_by_rating_range.max()

print(f"Restaurant rating range with the highest total revenue: {highest_revenue_range}")
print(f"Total revenue for this range: {max_revenue_rating_range:.2f}")

Restaurant rating range with the highest total revenue: 4.6-5.0
Total revenue for this range: 2197030.75


In [30]:
bins = [3.0, 3.6, 4.1, 4.6, 5.1] # Define the boundaries for rating bins
labels = ['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0'] # Define labels for each rating range

merged_df['rating_range'] = pd.cut(merged_df['rating'], bins=bins, labels=labels, right=False) # Create a new column 'rating_range' based on rating bins

revenue_by_rating_range = merged_df.groupby('rating_range', observed=False)['total_amount'].sum() # Group by rating range and sum total_amount, including all categories
highest_revenue_range = revenue_by_rating_range.idxmax() # Find the rating range with the highest total revenue
max_revenue_rating_range = revenue_by_rating_range.max() # Get the maximum revenue for that range

print(f"Restaurant rating range with the highest total revenue: {highest_revenue_range}") # Print the highest revenue rating range
print(f"Total revenue for this range: {max_revenue_rating_range:.2f}") # Print the total revenue for that range

Restaurant rating range with the highest total revenue: 4.6-5.0
Total revenue for this range: 2197030.75


In [31]:
gold_members_df = merged_df[merged_df['membership'] == 'Gold'] # Filter the DataFrame for 'Gold' members
city_avg_order_value_gold = gold_members_df.groupby('city')['total_amount'].mean() # Group by city and calculate the average 'total_amount'
highest_avg_city_gold = city_avg_order_value_gold.idxmax() # Find the city with the highest average order value among Gold members
max_avg_value_gold = city_avg_order_value_gold.max() # Get the highest average order value

print(f"City among Gold members with the highest average order value: {highest_avg_city_gold}") # Print the city name
print(f"Highest average order value for Gold members in {highest_avg_city_gold}: {max_avg_value_gold:.2f}") # Print the average order value

City among Gold members with the highest average order value: Chennai
Highest average order value for Gold members in Chennai: 808.46


In [32]:
cuisine_summary = merged_df.groupby('cuisine').agg( # Group by cuisine to get aggregated statistics
    distinct_restaurants=('restaurant_id', 'nunique'), # Count unique restaurants per cuisine
    total_revenue=('total_amount', 'sum') # Sum total revenue per cuisine
).reset_index() # Reset index to make 'cuisine' a regular column

total_overall_revenue = cuisine_summary['total_revenue'].sum() # Calculate the sum of all revenues
significant_revenue_threshold = total_overall_revenue * 0.05 # Set a threshold for 'significant revenue' (e.g., 5% of total)

significant_cuisines = cuisine_summary[cuisine_summary['total_revenue'] >= significant_revenue_threshold] # Filter for cuisines that meet the revenue threshold

if not significant_cuisines.empty: # Check if any cuisines meet the criteria
    target_cuisine = significant_cuisines.loc[significant_cuisines['distinct_restaurants'].idxmin()] # Find the cuisine with the lowest restaurant count among significant ones
    print(f"Cuisine with lowest distinct restaurants and significant revenue: {target_cuisine['cuisine']}") # Print the cuisine name
    print(f"Number of distinct restaurants: {target_cuisine['distinct_restaurants']}") # Print the count of distinct restaurants
    print(f"Total revenue: {target_cuisine['total_revenue']:.2f}") # Print the total revenue
else: # If no cuisines meet the threshold
    print("No cuisine meets the significant revenue threshold.") # Inform that no suitable cuisine was found

Cuisine with lowest distinct restaurants and significant revenue: Chinese
Number of distinct restaurants: 120
Total revenue: 1930504.65


In [33]:
total_orders = len(merged_df) # Get the total number of orders in the DataFrame
gold_member_orders = merged_df[merged_df['membership'] == 'Gold'].shape[0] # Count orders placed by 'Gold' members

percentage_gold_members = round((gold_member_orders / total_orders) * 100) # Calculate the percentage and round to the nearest integer

print(f"Total orders: {total_orders}") # Print the total number of orders
print(f"Orders by Gold members: {gold_member_orders}") # Print the number of orders by Gold members
print(f"Percentage of total orders placed by Gold members: {percentage_gold_members:.0f}%") # Print the calculated percentage

Total orders: 10000
Orders by Gold members: 4987
Percentage of total orders placed by Gold members: 50%


In [34]:
restaurant_summary = merged_df.groupby('restaurant_name_y').agg( # Group data by restaurant name
    average_order_value=('total_amount', 'mean'), # Calculate the average order value
    order_count=('order_id', 'count') # Count the number of orders for each restaurant
).reset_index() # Reset index to make restaurant name a regular column

restaurants_less_than_20_orders = restaurant_summary[restaurant_summary['order_count'] < 20] # Filter for restaurants with less than 20 orders

if not restaurants_less_than_20_orders.empty: # Check if there are any restaurants meeting the criteria
    highest_avg_restaurant = restaurants_less_than_20_orders.loc[restaurants_less_than_20_orders['average_order_value'].idxmax()] # Find the restaurant with the highest average order value
    print(f"Restaurant with the highest average order value and less than 20 orders: {highest_avg_restaurant['restaurant_name_y']}") # Print the restaurant name
    print(f"Average order value: {highest_avg_restaurant['average_order_value']:.2f}") # Print its average order value
    print(f"Number of orders: {highest_avg_restaurant['order_count']}") # Print its order count
else: # If no such restaurants exist
    print("No restaurants found with less than 20 orders.") # Inform that no suitable restaurant was found

Restaurant with the highest average order value and less than 20 orders: Restaurant_294
Average order value: 1040.22
Number of orders: 13


In [35]:
gold_indian_revenue = merged_df[(merged_df['membership'] == 'Gold') & (merged_df['cuisine'] == 'Indian')]['total_amount'].sum() # Calculate revenue for Gold + Indian
gold_italian_revenue = merged_df[(merged_df['membership'] == 'Gold') & (merged_df['cuisine'] == 'Italian')]['total_amount'].sum() # Calculate revenue for Gold + Italian
regular_indian_revenue = merged_df[(merged_df['membership'] == 'Regular') & (merged_df['cuisine'] == 'Indian')]['total_amount'].sum() # Calculate revenue for Regular + Indian
regular_chinese_revenue = merged_df[(merged_df['membership'] == 'Regular') & (merged_df['cuisine'] == 'Chinese')]['total_amount'].sum() # Calculate revenue for Regular + Chinese

revenue_combinations = { # Store calculated revenues in a dictionary
    'Gold + Indian': gold_indian_revenue,
    'Gold + Italian': gold_italian_revenue,
    'Regular + Indian': regular_indian_revenue,
    'Regular + Chinese': regular_chinese_revenue
}

highest_revenue_combination = max(revenue_combinations, key=revenue_combinations.get) # Find the combination with the highest revenue
max_revenue_value = revenue_combinations[highest_revenue_combination] # Get the maximum revenue value

print(f"Combination with the highest revenue: {highest_revenue_combination}") # Print the highest revenue combination
print(f"Total revenue for this combination: {max_revenue_value:.2f}") # Print the total revenue for that combination

Combination with the highest revenue: Gold + Italian
Total revenue for this combination: 1005779.05


In [36]:
merged_df['order_date'] = pd.to_datetime(merged_df['order_date'], format='%d-%m-%Y') # Convert 'order_date' column to datetime objects
merged_df['quarter'] = merged_df['order_date'].dt.quarter # Extract the quarter of the year from the 'order_date'

revenue_by_quarter = merged_df.groupby('quarter')['total_amount'].sum() # Group by quarter and sum the 'total_amount'
highest_revenue_quarter = revenue_by_quarter.idxmax() # Find the quarter with the highest total revenue
max_revenue_quarter = revenue_by_quarter.max() # Get the maximum revenue for that quarter

print(f"Quarter of the year with the highest total revenue: Q{highest_revenue_quarter}") # Print the quarter with highest revenue
print(f"Total revenue for Q{highest_revenue_quarter}: {max_revenue_quarter:.2f}") # Print the total revenue for that quarter

Quarter of the year with the highest total revenue: Q3
Total revenue for Q3: 2037385.10


In [37]:
gold_members_orders = merged_df[merged_df['membership'] == 'Gold'] # Filter the DataFrame to get orders from 'Gold' members
total_gold_orders = len(gold_members_orders) # Count the number of rows (orders) in the filtered DataFrame

print(f"Total number of orders placed by Gold members: {total_gold_orders}") # Print the total count

Total number of orders placed by Gold members: 4987


In [38]:
hyderabad_orders = merged_df[merged_df['city'] == 'Hyderabad'] # Filter the DataFrame for orders placed in 'Hyderabad'
total_revenue_hyderabad = hyderabad_orders['total_amount'].sum() # Calculate the sum of 'total_amount'
rounded_revenue_hyderabad = round(total_revenue_hyderabad) # Round the total revenue to the nearest integer

print(f"Total revenue generated from orders in Hyderabad city: ₹{rounded_revenue_hyderabad:.0f}") # Print the result

Total revenue generated from orders in Hyderabad city: ₹1889367


In [39]:
distinct_users_count = merged_df['user_id'].nunique() # Count the number of unique 'user_id' values
print(f"Number of distinct users who have placed at least one order: {distinct_users_count}") # Print the count

Number of distinct users who have placed at least one order: 2883


In [40]:
gold_members_df = merged_df[merged_df['membership'] == 'Gold'] # Filter the DataFrame for 'Gold' members
average_order_value_gold = gold_members_df['total_amount'].mean() # Calculate the average 'total_amount' for Gold members

print(f"Average order value for Gold members: ₹{average_order_value_gold:.2f}") # Print the result, rounded to two decimal places

Average order value for Gold members: ₹797.15


In [41]:
high_rated_restaurants_orders = merged_df[merged_df['rating'] >= 4.5] # Filter for orders from restaurants with a rating of 4.5 or higher
orders_count_high_rating = len(high_rated_restaurants_orders) # Count the number of such orders

print(f"Number of orders placed for restaurants with a rating of 4.5 or higher: {orders_count_high_rating}") # Print the final count

Number of orders placed for restaurants with a rating of 4.5 or higher: 3374


In [42]:
orders_in_top_gold_city = gold_members_df[gold_members_df['city'] == highest_revenue_city] # Filter Gold member orders for the top revenue city
count_orders_top_gold_city = len(orders_in_top_gold_city) # Count the number of orders in the filtered DataFrame

print(f"Number of orders placed by Gold members in {highest_revenue_city}: {count_orders_top_gold_city}") # Print the result

Number of orders placed by Gold members in Chennai: 1337
